In [6]:
import os
from dotenv import load_dotenv 
load_dotenv(override=True)

DeepSeek_API_KEY = os.getenv("DEEPSEEK_API_KEY")
# print(DeepSeek_API_KEY)  # 可以通过打印查看

### 初始化模型

In [7]:
from langchain.chat_models import init_chat_model

model = init_chat_model(model="deepseek-chat", model_provider="deepseek")  

In [3]:
question = "你好，请你介绍一下你自己。"

result = model.invoke(question)
print(result.content)

你好！很高兴认识你！😊

我是DeepSeek，由深度求索公司创造的AI助手。让我来详细介绍一下自己：

**基本信息：**
- 我是一个纯文本AI模型，专注于理解和生成自然语言
- 知识截止到2024年7月，是DeepSeek的最新版本

**我的能力：**
- 💬 对话交流，回答各种问题
- 📝 文本创作、编辑、翻译
- 🔍 知识问答和分析
- 📚 学习辅导和解题
- 📄 文件处理 - 支持上传图像、txt、pdf、ppt、word、excel文件，并从中读取文字信息
- 🌐 联网搜索（需要你在Web/App手动开启）

**使用方式：**
- 完全免费使用，没有任何收费计划
- 上下文长度128K，可以处理很长的对话
- 可以通过官方应用商店下载App使用

**我的特点：**
- 热情细腻的交流风格
- 乐于助人，尽力提供准确有用的回答
- 会诚实地承认不知道的事情

有什么问题或者想聊的话题吗？我很乐意帮助你！✨


In [4]:
result.additional_kwargs

{'refusal': None}

### 创建LangGraph智能体

In [1]:
from langchain_core.tools import tool
from pydantic import BaseModel, Field
import requests,json

class WeatherQuery(BaseModel):
    loc: str = Field(description="The location name of the city")

@tool(args_schema = WeatherQuery)
def get_weather(loc):
    """
    查询即时天气函数
    :param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，\
    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入'Beijing'；
    :return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather\
    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息
    """
    # Step 1.构建请求
    url = "https://api.openweathermap.org/data/2.5/weather"

    # Step 2.设置查询参数
    params = {
        "q": loc,               
        "appid": os.getenv("OPENWEATHER_API_KEY"),    # 输入API key
        "units": "metric",            # 使用摄氏度而不是华氏度
        "lang":"zh_cn"                # 输出语言为简体中文
    }

    # Step 3.发送GET请求
    response = requests.get(url, params=params)
    
    # Step 4.解析响应
    data = response.json()
    return json.dumps(data)

In [2]:
print(f'''
name: {get_weather.name}
description: {get_weather.description}
arguments: {get_weather.args}
''')


name: get_weather
description: 查询即时天气函数
:param loc: 必要参数，字符串类型，用于表示查询天气的具体城市名称，    注意，中国的城市需要用对应城市的英文名称代替，例如如果需要查询北京市天气，则loc参数需要输入'Beijing'；
:return：OpenWeather API查询即时天气的结果，具体URL请求地址为：https://api.openweathermap.org/data/2.5/weather    返回结果对象类型为解析之后的JSON格式对象，并用字符串形式进行表示，其中包含了全部重要的天气信息
arguments: {'loc': {'description': 'The location name of the city', 'title': 'Loc', 'type': 'string'}}



In [3]:
# 封装外部函数列表
tools = [get_weather]

In [4]:
from langgraph.prebuilt import create_react_agent

In [8]:
agent = create_react_agent(model=model, tools=tools)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_14908\1452437597.py:1: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(model=model, tools=tools)


In [9]:
response = agent.invoke({"messages": [{"role": "user", "content": "你好，请介绍下你自己。"}]})

In [ ]:
response

{'messages': [HumanMessage(content='你好，请介绍下你自己。', additional_kwargs={}, response_metadata={}, id='4584f8b2-8fec-441a-8afc-06d5cbea0b3a'),
  AIMessage(content='你好！我是一个AI助手，很高兴为你服务！\n\n我可以帮助你处理各种任务，包括：\n- 查询天气信息\n- 回答各种问题\n- 协助处理文本内容\n- 提供信息和建议\n\n目前我特别擅长查询天气信息，可以为你提供全球各个城市的实时天气情况。如果你需要查询某个城市的天气，只需要告诉我城市名称（中国的城市请用英文名称），我就能为你获取最新的天气数据。\n\n有什么我可以帮助你的吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 94, 'prompt_tokens': 265, 'total_tokens': 359, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 265}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': '9501507b-5345-4213-b75f-e3d8a068431f', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--e4e863ac-4702-4a62-819b-942815dadfdd-0', usage_metadata={'input_tokens': 265, 'output_tokens': 94, 'total_tokens': 359, 'inp

In [ ]:
response["messages"]

[HumanMessage(content='你好，请介绍下你自己。', additional_kwargs={}, response_metadata={}, id='4584f8b2-8fec-441a-8afc-06d5cbea0b3a'),
 AIMessage(content='你好！我是一个AI助手，很高兴为你服务！\n\n我可以帮助你处理各种任务，包括：\n- 查询天气信息\n- 回答各种问题\n- 协助处理文本内容\n- 提供信息和建议\n\n目前我特别擅长查询天气信息，可以为你提供全球各个城市的实时天气情况。如果你需要查询某个城市的天气，只需要告诉我城市名称（中国的城市请用英文名称），我就能为你获取最新的天气数据。\n\n有什么我可以帮助你的吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 94, 'prompt_tokens': 265, 'total_tokens': 359, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 265}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': '9501507b-5345-4213-b75f-e3d8a068431f', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--e4e863ac-4702-4a62-819b-942815dadfdd-0', usage_metadata={'input_tokens': 265, 'output_tokens': 94, 'total_tokens': 359, 'input_token_detai

In [ ]:
print(response["messages"][-1].content)

你好！我是一个AI助手，很高兴为你服务！

我可以帮助你处理各种任务，包括：
- 查询天气信息
- 回答各种问题
- 协助处理文本内容
- 提供信息和建议

目前我特别擅长查询天气信息，可以为你提供全球各个城市的实时天气情况。如果你需要查询某个城市的天气，只需要告诉我城市名称（中国的城市请用英文名称），我就能为你获取最新的天气数据。

有什么我可以帮助你的吗？


In [16]:
response = agent.invoke({"messages": [{"role": "user", "content": "请问北京今天天气如何？"}]})
print(response["messages"][-1].content)

根据查询结果，北京今天的天气情况如下：

**天气状况**：🌤️ 晴，少云
**温度**：11.9°C
**体感温度**：10.3°C
**湿度**：41%
**气压**：1018 hPa
**风速**：1.25 m/s
**能见度**：10公里

总体来说，今天北京天气不错，晴朗少云，温度适中，风力较小，是个比较舒适的日子。


In [11]:
response['messages']

[HumanMessage(content='请问北京今天天气如何？', additional_kwargs={}, response_metadata={}, id='651e9728-1dc8-4c70-98df-8b97d7d77dfe'),
 AIMessage(content='我来帮您查询北京今天的天气情况。', additional_kwargs={'tool_calls': [{'id': 'call_00_q9F9zKokvt208GxJELLIyzKX', 'function': {'arguments': '{"loc": "Beijing"}', 'name': 'get_weather'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 264, 'total_tokens': 287, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 256}, 'prompt_cache_hit_tokens': 256, 'prompt_cache_miss_tokens': 8}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': '08dfe276-7e7b-4b67-9b0f-24efaf90e0a9', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--553cdc4e-08a4-4986-beab-be4a10e7b5f8-0', tool_calls=[{'name': 'get_weather', 'args': {'loc': 'Beijing'}, 'id': 'call_00_q9F9zKokvt208GxJELLIy

In [12]:
len(response['messages'])

4

In [13]:
response['messages'][0]

HumanMessage(content='请问北京今天天气如何？', additional_kwargs={}, response_metadata={}, id='651e9728-1dc8-4c70-98df-8b97d7d77dfe')

In [14]:
response['messages'][1]

AIMessage(content='我来帮您查询北京今天的天气情况。', additional_kwargs={'tool_calls': [{'id': 'call_00_q9F9zKokvt208GxJELLIyzKX', 'function': {'arguments': '{"loc": "Beijing"}', 'name': 'get_weather'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 264, 'total_tokens': 287, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 256}, 'prompt_cache_hit_tokens': 256, 'prompt_cache_miss_tokens': 8}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': '08dfe276-7e7b-4b67-9b0f-24efaf90e0a9', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--553cdc4e-08a4-4986-beab-be4a10e7b5f8-0', tool_calls=[{'name': 'get_weather', 'args': {'loc': 'Beijing'}, 'id': 'call_00_q9F9zKokvt208GxJELLIyzKX', 'type': 'tool_call'}], usage_metadata={'input_tokens': 264, 'output_tokens': 23, 'total_tokens': 287, 'input_token_detai

In [15]:
response['messages'][2]

ToolMessage(content='{"coord": {"lon": 116.3972, "lat": 39.9075}, "weather": [{"id": 801, "main": "Clouds", "description": "\\u6674\\uff0c\\u5c11\\u4e91", "icon": "02d"}], "base": "stations", "main": {"temp": 11.94, "feels_like": 10.26, "temp_min": 11.94, "temp_max": 11.94, "pressure": 1018, "humidity": 41, "sea_level": 1018, "grnd_level": 1013}, "visibility": 10000, "wind": {"speed": 1.25, "deg": 241, "gust": 2.34}, "clouds": {"all": 14}, "dt": 1761722178, "sys": {"type": 1, "id": 9609, "country": "CN", "sunrise": 1761691169, "sunset": 1761729396}, "timezone": 28800, "id": 1816670, "name": "Beijing", "cod": 200}', name='get_weather', id='75af2554-71df-4f3f-b060-86c5781de88d', tool_call_id='call_00_q9F9zKokvt208GxJELLIyzKX')